# 🛠️ 05: Creating Custom Tools

Learn how to create your own tools to extend your LLM's capabilities with custom functionality tailored to your needs.

## 📋 Learning Objectives

By the end of this notebook, you will be able to:

- [ ] Create custom tools using the `@tool` decorator
- [ ] Define tool parameters with proper type hints
- [ ] Write clear tool descriptions for the LLM
- [ ] Register custom tools with the client
- [ ] Handle errors gracefully in tool functions
- [ ] Follow best practices for tool design
- [ ] Build a complete unit converter tool

## 🎯 Prerequisites

- Completed notebook 04 (Tool Calling Basics)
- Understanding of built-in tools and tool registration
- Basic Python knowledge (functions, type hints, decorators)

## ⏱️ Estimated Time: 15 minutes

## 1️⃣ Your First Custom Tool

The simplest way to create a tool is with the `@tool` decorator.

In [1]:
from local_llm_sdk import LocalLLMClient
from local_llm_sdk.tools import tool

# Create a simple greeting tool
@tool("Generates a personalized greeting for a given name")
def greet(name: str) -> dict:
    """Greet a person by name.
    
    Args:
        name: The person's name to greet
        
    Returns:
        A dictionary with the greeting message
    """
    return {
        "greeting": f"Hello, {name}! Welcome to custom tools!",
        "name": name
    }

print("✅ Created custom tool: greet")

⚠ Warning: Could not auto-detect model (HTTPConnectionPool(host='localhost', port=1234): Max retries exceeded with url: /v1/models (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7e0f24f10e90>: Failed to establish a new connection: [Errno 111] Connection refused'))). You'll need to specify model per request.
✅ Created custom tool: greet


**💡 Anatomy of a tool:**

1. **`@tool(description)`**: Decorator that registers the function
   - Description tells the LLM when to use this tool
   
2. **Type hints**: `name: str` and `-> dict`
   - Required! Tells the LLM what parameters to provide
   
3. **Docstring**: Detailed documentation (optional but recommended)
   - Helps the LLM understand parameter meanings
   
4. **Return dict**: Tools should return dictionaries
   - Makes results easy to parse and display

Now let's register and use it:

In [3]:
# Create client and register the tool
client = LocalLLMClient(
    base_url="http://169.254.83.107:1234/v1",
    model="mistralai/magistral-small-2509"
)

client.register_tools([greet])

# Use the tool
response = client.chat("Please greet Alice")
print(response)
client.print_tool_calls()

Hello, Alice! Welcome to custom tools!

🔧 Tool Execution Summary (1 call):
  [1] greet(name=Alice) → greeting=Hello, Alice! Welcome to custom tools!



**🎉 You just created and used your first custom tool!**

## 2️⃣ Multiple Parameters

Tools can have multiple parameters of different types.

In [5]:
@tool("Calculates the area of a rectangle given width and height")
def rectangle_area(width: float, height: float) -> dict:
    """Calculate rectangle area.
    
    Args:
        width: The width of the rectangle
        height: The height of the rectangle
        
    Returns:
        Dictionary with area, perimeter, and dimensions
    """
    area = width * height
    perimeter = 2 * (width + height)
    
    return {
        "area": area,
        "perimeter": perimeter,
        "width": width,
        "height": height,
        "unit": "square units"
    }

# Register and use
client.register_tools([rectangle_area])

response = client.chat("What's the area and perimeter of a rectangle that is 12.5 by 8.3?")
print(response)
client.print_tool_calls()

The area of the rectangle is 103.75 square units and the perimeter is 41.6 units.

🔧 Tool Execution Summary (1 call):
  [1] rectangle_area(width=12.5, height=8.3) → area=103.75000000000001



**💡 Supported parameter types:**
- `str`: Text strings
- `int`: Whole numbers
- `float`: Decimal numbers
- `bool`: True/False values
- `list`: Arrays (with type hints like `list[str]`)
- `dict`: Objects/maps

## 3️⃣ Optional Parameters with Defaults

You can provide default values for optional parameters.

In [7]:
@tool("Formats a price with currency symbol and optional decimal places")
def format_price(amount: float, currency: str = "USD", decimals: int = 2) -> dict:
    """Format a price with currency symbol.
    
    Args:
        amount: The price amount
        currency: Currency code (USD, EUR, GBP, etc.). Default: USD
        decimals: Number of decimal places. Default: 2
        
    Returns:
        Dictionary with formatted price string
    """
    symbols = {
        "USD": "$",
        "EUR": "€",
        "GBP": "£",
        "JPY": "¥"
    }
    
    symbol = symbols.get(currency, currency)
    formatted = f"{symbol}{amount:.{decimals}f}"
    
    return {
        "formatted_price": formatted,
        "amount": amount,
        "currency": currency
    }

client.register_tools([format_price])

# Test with different parameter combinations
print("Test 1: Default parameters")
response1 = client.chat("Format the price 1234.567")
print(response1)
client.print_tool_calls()

print("\nTest 2: Custom currency")
response2 = client.chat("Format 99.99 in euros")
print(response2)
client.print_tool_calls()

print("\nTest 3: Custom decimals")
response3 = client.chat("Format 1500 yen with no decimal places")
print(response3)
client.print_tool_calls()

Test 1: Default parameters
The formatted price for $1234.567 is: **$1234.57**

🔧 Tool Execution Summary (1 call):
  [1] format_price(amount=1234.567) → formatted_price=$1234.57


Test 2: Custom currency
The formatted price is €99.99.

🔧 Tool Execution Summary (1 call):
  [1] format_price(amount=99.99, currency=EUR) → formatted_price=€99.99


Test 3: Custom decimals
The formatted price is ¥1500.

🔧 Tool Execution Summary (1 call):
  [1] format_price(amount=1500, currency=JPY, decimals=0) → formatted_price=¥1500



## 4️⃣ Error Handling

Always handle errors gracefully in your tools.

In [8]:
@tool("Divides two numbers safely, handling division by zero")
def safe_divide(numerator: float, denominator: float) -> dict:
    """Safely divide two numbers.
    
    Args:
        numerator: The number to divide
        denominator: The number to divide by
        
    Returns:
        Dictionary with result or error message
    """
    if denominator == 0:
        return {
            "error": "Cannot divide by zero",
            "numerator": numerator,
            "denominator": denominator,
            "result": None
        }
    
    result = numerator / denominator
    return {
        "result": result,
        "numerator": numerator,
        "denominator": denominator,
        "error": None
    }

client.register_tools([safe_divide])

# Test normal case
print("Normal division:")
response1 = client.chat("Divide 100 by 4")
print(response1)

# Test error case
print("\nDivision by zero:")
response2 = client.chat("What is 50 divided by 0?")
print(response2)

Normal division:
The result of dividing 100 by 4 is 25.0.

Division by zero:
I'm unable to provide a result for dividing 50 by 0 because division by zero is undefined in mathematics. The operation 50 ÷ 0 is not allowed.


**✅ Best practices for error handling:**

1. **Return errors in the result dict** (don't raise exceptions)
2. **Include an `error` field** in the return dict
3. **Provide helpful error messages** so the LLM can explain to the user
4. **Return partial results** when possible (even if there's an error)
5. **Validate inputs** before processing

## 5️⃣ Complex Tool: String Analysis

Let's build a more sophisticated tool that performs multiple analyses.

In [9]:
@tool("Performs comprehensive analysis on a text string including length, word count, and character breakdown")
def analyze_text(text: str, include_details: bool = True) -> dict:
    """Analyze text comprehensively.
    
    Args:
        text: The text to analyze
        include_details: Whether to include detailed character breakdown
        
    Returns:
        Dictionary with various text statistics
    """
    # Basic stats
    char_count = len(text)
    char_count_no_spaces = len(text.replace(" ", ""))
    word_count = len(text.split())
    sentence_count = text.count('.') + text.count('!') + text.count('?')
    
    result = {
        "text": text,
        "character_count": char_count,
        "character_count_no_spaces": char_count_no_spaces,
        "word_count": word_count,
        "sentence_count": sentence_count,
        "avg_word_length": round(char_count_no_spaces / word_count, 2) if word_count > 0 else 0
    }
    
    # Detailed breakdown (optional)
    if include_details:
        result["details"] = {
            "uppercase_count": sum(1 for c in text if c.isupper()),
            "lowercase_count": sum(1 for c in text if c.islower()),
            "digit_count": sum(1 for c in text if c.isdigit()),
            "space_count": text.count(' '),
            "special_char_count": sum(1 for c in text if not c.isalnum() and not c.isspace())
        }
    
    return result

client.register_tools([analyze_text])

# Test the analyzer
response = client.chat(
    "Analyze this text in detail: 'The Quick Brown Fox Jumps Over 13 Lazy Dogs!'"
)
print(response)
client.print_tool_calls()

The analysis of the text "'The Quick Brown Fox Jumps Over 13 Lazy Dogs!'" is as follows:

- **Character Count**: The total number of characters in the text is 46.
- **Character Count (without spaces)**: Excluding spaces, there are 38 characters.
- **Word Count**: There are 9 words in this sentence.
- **Sentence Count**: This is a single sentence.
- **Average Word Length**: The average word length is approximately 4.22 characters.

### Details:

- **Uppercase Letters**: There are 8 uppercase letters (T, Q, B, F, J, O, L, D).
- **Lowercase Letters**: There are 25 lowercase letters.
- **Digits**: There are 2 digits in the text (the number '13').
- **Spaces**: There are 8 spaces separating the words.
- **Special Characters**: There are 3 special characters: the exclamation mark (!), the apostrophe ('), and the space character.

This pangram is often used for typing practice as it contains every letter of the alphabet at least once.

🔧 Tool Execution Summary (1 call):
  [1] analyze_text(tex

## 6️⃣ Registering Multiple Tools at Once

You can register multiple custom tools together.

In [10]:
# Create several related tools
@tool("Converts temperature from Celsius to Fahrenheit")
def celsius_to_fahrenheit(celsius: float) -> dict:
    fahrenheit = (celsius * 9/5) + 32
    return {
        "celsius": celsius,
        "fahrenheit": fahrenheit,
        "conversion": f"{celsius}°C = {fahrenheit}°F"
    }

@tool("Converts temperature from Fahrenheit to Celsius")
def fahrenheit_to_celsius(fahrenheit: float) -> dict:
    celsius = (fahrenheit - 32) * 5/9
    return {
        "fahrenheit": fahrenheit,
        "celsius": celsius,
        "conversion": f"{fahrenheit}°F = {celsius}°C"
    }

@tool("Converts distance from kilometers to miles")
def km_to_miles(kilometers: float) -> dict:
    miles = kilometers * 0.621371
    return {
        "kilometers": kilometers,
        "miles": round(miles, 2),
        "conversion": f"{kilometers} km = {round(miles, 2)} miles"
    }

@tool("Converts distance from miles to kilometers")
def miles_to_km(miles: float) -> dict:
    kilometers = miles * 1.60934
    return {
        "miles": miles,
        "kilometers": round(kilometers, 2),
        "conversion": f"{miles} miles = {round(kilometers, 2)} km"
    }

# Register all conversion tools
conversion_tools = [
    celsius_to_fahrenheit,
    fahrenheit_to_celsius,
    km_to_miles,
    miles_to_km
]

client.register_tools(conversion_tools)

print("✅ Registered 4 conversion tools")
print("\nAvailable tools:")
for tool_name in client.tools.list_tools():
    print(f"  - {tool_name}")

✅ Registered 4 conversion tools

Available tools:
  - greet
  - rectangle_area
  - format_price
  - safe_divide
  - analyze_text
  - celsius_to_fahrenheit
  - fahrenheit_to_celsius
  - km_to_miles
  - miles_to_km


Test the conversion tools:

In [11]:
# Temperature conversions
print("Temperature conversions:")
response1 = client.chat("What is 25 degrees Celsius in Fahrenheit?")
print(f"  {response1}")
client.print_tool_calls()

response2 = client.chat("Convert 98.6°F to Celsius")
print(f"  {response2}")
client.print_tool_calls()

# Distance conversions
print("\nDistance conversions:")
response3 = client.chat("How many miles is 100 kilometers?")
print(f"  {response3}")
client.print_tool_calls()

response4 = client.chat("Convert 26.2 miles to kilometers")
print(f"  {response4}")
client.print_tool_calls()

Temperature conversions:
  25 degrees Celsius is equal to 77.0 degrees Fahrenheit.

🔧 Tool Execution Summary (1 call):
  [1] celsius_to_fahrenheit(celsius=25) → celsius=25

  The conversion of 98.6°F to Celsius is:

98.6°F = 37.0°C

🔧 Tool Execution Summary (1 call):
  [1] fahrenheit_to_celsius(fahrenheit=98.6) → fahrenheit=98.6


Distance conversions:
  100 kilometers is approximately 62.14 miles.

🔧 Tool Execution Summary (1 call):
  [1] km_to_miles(kilometers=100) → kilometers=100

  The conversion of 26.2 miles to kilometers is approximately 42.16 kilometers. This is calculated using the conversion factor where 1 mile equals 1.60934 kilometers, so 26.2 miles * 1.60934 = 42.16 km.

🔧 Tool Execution Summary (1 call):
  [1] miles_to_km(miles=26.2) → miles=26.2



## 🏋️ Exercise: Complete Unit Converter

**Challenge:** Create a comprehensive unit converter tool that handles:
1. Temperature conversions (Celsius, Fahrenheit, Kelvin)
2. Distance conversions (meters, kilometers, miles, feet)
3. Weight conversions (grams, kilograms, pounds, ounces)

Requirements:
- Use a single tool with parameters: `value`, `from_unit`, `to_unit`
- Support at least 3 unit types
- Handle invalid unit conversions with error messages
- Return formatted conversion string

Try it yourself first!

In [ ]:
# Your code here:



<details>
<summary>Click to see solution</summary>

```python
# Solution: Comprehensive unit converter

@tool("Converts between different units of temperature, distance, and weight")
def convert_units(value: float, from_unit: str, to_unit: str) -> dict:
    """Convert between various units.
    
    Args:
        value: The numeric value to convert
        from_unit: Source unit (celsius, fahrenheit, kelvin, km, miles, meters, feet, kg, pounds, grams, ounces)
        to_unit: Target unit (same options as from_unit)
        
    Returns:
        Dictionary with converted value or error
    """
    # Normalize units to lowercase
    from_unit = from_unit.lower()
    to_unit = to_unit.lower()
    
    # Temperature conversions
    if from_unit in ['celsius', 'c'] and to_unit in ['fahrenheit', 'f']:
        result = (value * 9/5) + 32
        return {"value": result, "unit": "Fahrenheit", "conversion": f"{value}°C = {result}°F"}
    
    elif from_unit in ['fahrenheit', 'f'] and to_unit in ['celsius', 'c']:
        result = (value - 32) * 5/9
        return {"value": result, "unit": "Celsius", "conversion": f"{value}°F = {result:.2f}°C"}
    
    elif from_unit in ['celsius', 'c'] and to_unit in ['kelvin', 'k']:
        result = value + 273.15
        return {"value": result, "unit": "Kelvin", "conversion": f"{value}°C = {result}K"}
    
    elif from_unit in ['kelvin', 'k'] and to_unit in ['celsius', 'c']:
        result = value - 273.15
        return {"value": result, "unit": "Celsius", "conversion": f"{value}K = {result:.2f}°C"}
    
    elif from_unit in ['fahrenheit', 'f'] and to_unit in ['kelvin', 'k']:
        result = (value - 32) * 5/9 + 273.15
        return {"value": result, "unit": "Kelvin", "conversion": f"{value}°F = {result:.2f}K"}
    
    elif from_unit in ['kelvin', 'k'] and to_unit in ['fahrenheit', 'f']:
        result = (value - 273.15) * 9/5 + 32
        return {"value": result, "unit": "Fahrenheit", "conversion": f"{value}K = {result:.2f}°F"}
    
    # Distance conversions
    elif from_unit in ['kilometers', 'km'] and to_unit in ['miles', 'mi']:
        result = value * 0.621371
        return {"value": result, "unit": "miles", "conversion": f"{value} km = {result:.2f} miles"}
    
    elif from_unit in ['miles', 'mi'] and to_unit in ['kilometers', 'km']:
        result = value * 1.60934
        return {"value": result, "unit": "kilometers", "conversion": f"{value} miles = {result:.2f} km"}
    
    elif from_unit in ['meters', 'm'] and to_unit in ['feet', 'ft']:
        result = value * 3.28084
        return {"value": result, "unit": "feet", "conversion": f"{value} m = {result:.2f} ft"}
    
    elif from_unit in ['feet', 'ft'] and to_unit in ['meters', 'm']:
        result = value * 0.3048
        return {"value": result, "unit": "meters", "conversion": f"{value} ft = {result:.2f} m"}
    
    # Weight conversions
    elif from_unit in ['kilograms', 'kg'] and to_unit in ['pounds', 'lbs', 'lb']:
        result = value * 2.20462
        return {"value": result, "unit": "pounds", "conversion": f"{value} kg = {result:.2f} lbs"}
    
    elif from_unit in ['pounds', 'lbs', 'lb'] and to_unit in ['kilograms', 'kg']:
        result = value * 0.453592
        return {"value": result, "unit": "kilograms", "conversion": f"{value} lbs = {result:.2f} kg"}
    
    elif from_unit in ['grams', 'g'] and to_unit in ['ounces', 'oz']:
        result = value * 0.035274
        return {"value": result, "unit": "ounces", "conversion": f"{value} g = {result:.2f} oz"}
    
    elif from_unit in ['ounces', 'oz'] and to_unit in ['grams', 'g']:
        result = value * 28.3495
        return {"value": result, "unit": "grams", "conversion": f"{value} oz = {result:.2f} g"}
    
    # Error: unsupported conversion
    else:
        return {
            "error": f"Unsupported conversion: {from_unit} to {to_unit}",
            "value": None,
            "supported_units": "celsius, fahrenheit, kelvin, km, miles, meters, feet, kg, pounds, grams, ounces"
        }

# Register and test
client.register_tools([convert_units])

print("🔄 Universal Unit Converter\n")
print("="*70 + "\n")

# Test various conversions
tests = [
    "Convert 100 celsius to fahrenheit",
    "What is 0 kelvin in celsius?",
    "How many miles is 42 kilometers?",
    "Convert 6 feet to meters",
    "What's 70 kilograms in pounds?",
    "How many ounces is 500 grams?"
]

for test in tests:
    print(f"Q: {test}")
    response = client.chat(test)
    print(f"A: {response}\n")
```
</details>

In [ ]:
# Solution cell (run this to see the answer)
@tool("Converts between different units of temperature, distance, and weight")
def convert_units(value: float, from_unit: str, to_unit: str) -> dict:
    """Convert between various units.
    
    Args:
        value: The numeric value to convert
        from_unit: Source unit (celsius, fahrenheit, kelvin, km, miles, meters, feet, kg, pounds, grams, ounces)
        to_unit: Target unit (same options as from_unit)
        
    Returns:
        Dictionary with converted value or error
    """
    from_unit = from_unit.lower()
    to_unit = to_unit.lower()
    
    # Temperature conversions
    if from_unit in ['celsius', 'c'] and to_unit in ['fahrenheit', 'f']:
        result = (value * 9/5) + 32
        return {"value": result, "unit": "Fahrenheit", "conversion": f"{value}°C = {result}°F"}
    elif from_unit in ['fahrenheit', 'f'] and to_unit in ['celsius', 'c']:
        result = (value - 32) * 5/9
        return {"value": result, "unit": "Celsius", "conversion": f"{value}°F = {result:.2f}°C"}
    elif from_unit in ['celsius', 'c'] and to_unit in ['kelvin', 'k']:
        result = value + 273.15
        return {"value": result, "unit": "Kelvin", "conversion": f"{value}°C = {result}K"}
    elif from_unit in ['kelvin', 'k'] and to_unit in ['celsius', 'c']:
        result = value - 273.15
        return {"value": result, "unit": "Celsius", "conversion": f"{value}K = {result:.2f}°C"}
    
    # Distance conversions
    elif from_unit in ['kilometers', 'km'] and to_unit in ['miles', 'mi']:
        result = value * 0.621371
        return {"value": result, "unit": "miles", "conversion": f"{value} km = {result:.2f} miles"}
    elif from_unit in ['miles', 'mi'] and to_unit in ['kilometers', 'km']:
        result = value * 1.60934
        return {"value": result, "unit": "kilometers", "conversion": f"{value} miles = {result:.2f} km"}
    elif from_unit in ['meters', 'm'] and to_unit in ['feet', 'ft']:
        result = value * 3.28084
        return {"value": result, "unit": "feet", "conversion": f"{value} m = {result:.2f} ft"}
    elif from_unit in ['feet', 'ft'] and to_unit in ['meters', 'm']:
        result = value * 0.3048
        return {"value": result, "unit": "meters", "conversion": f"{value} ft = {result:.2f} m"}
    
    # Weight conversions
    elif from_unit in ['kilograms', 'kg'] and to_unit in ['pounds', 'lbs', 'lb']:
        result = value * 2.20462
        return {"value": result, "unit": "pounds", "conversion": f"{value} kg = {result:.2f} lbs"}
    elif from_unit in ['pounds', 'lbs', 'lb'] and to_unit in ['kilograms', 'kg']:
        result = value * 0.453592
        return {"value": result, "unit": "kilograms", "conversion": f"{value} lbs = {result:.2f} kg"}
    elif from_unit in ['grams', 'g'] and to_unit in ['ounces', 'oz']:
        result = value * 0.035274
        return {"value": result, "unit": "ounces", "conversion": f"{value} g = {result:.2f} oz"}
    elif from_unit in ['ounces', 'oz'] and to_unit in ['grams', 'g']:
        result = value * 28.3495
        return {"value": result, "unit": "grams", "conversion": f"{value} oz = {result:.2f} g"}
    else:
        return {
            "error": f"Unsupported conversion: {from_unit} to {to_unit}",
            "value": None,
            "supported_units": "celsius, fahrenheit, kelvin, km, miles, meters, feet, kg, pounds, grams, ounces"
        }

client.register_tools([convert_units])

print("🔄 Universal Unit Converter\n")
print("="*70 + "\n")

tests = [
    "Convert 100 celsius to fahrenheit",
    "What is 0 kelvin in celsius?",
    "How many miles is 42 kilometers?",
    "Convert 6 feet to meters",
    "What's 70 kilograms in pounds?",
    "How many ounces is 500 grams?"
]

for test in tests:
    print(f"Q: {test}")
    response = client.chat(test)
    print(f"A: {response}\n")

## ⚠️ Common Pitfalls

### 1. Missing Type Hints
```python
# ❌ Bad: No type hints
@tool("Adds two numbers")
def add(a, b):  # LLM won't know parameter types!
    return {"result": a + b}

# ✅ Good: Clear type hints
@tool("Adds two numbers")
def add(a: float, b: float) -> dict:
    return {"result": a + b}
```

### 2. Poor Tool Descriptions
```python
# ❌ Bad: Vague description
@tool("Does something with text")
def process_text(text: str) -> dict:
    return {"result": text.upper()}

# ✅ Good: Specific description
@tool("Converts text to uppercase letters")
def uppercase_text(text: str) -> dict:
    return {"result": text.upper()}
```

### 3. Not Returning Dicts
```python
# ❌ Bad: Returns raw value
@tool("Calculates square")
def square(x: float) -> float:
    return x * x  # LLM has trouble parsing

# ✅ Good: Returns structured dict
@tool("Calculates square")
def square(x: float) -> dict:
    return {"input": x, "result": x * x}
```

### 4. Raising Exceptions
```python
# ❌ Bad: Raises exception
@tool("Gets array element")
def get_element(arr: list, index: int) -> dict:
    return {"element": arr[index]}  # Can crash!

# ✅ Good: Handles errors gracefully
@tool("Gets array element")
def get_element(arr: list, index: int) -> dict:
    if index < 0 or index >= len(arr):
        return {"error": "Index out of bounds", "element": None}
    return {"element": arr[index], "error": None}
```

### 5. Too Many Parameters
```python
# ⚠️ Warning: Too complex
@tool("Complex function with many parameters")
def complex_func(a: int, b: int, c: int, d: int, e: int, f: int) -> dict:
    # LLMs struggle with many parameters
    pass

# ✅ Better: Group into structured params
@tool("Processes configuration")
def process_config(config: dict) -> dict:
    # Or break into multiple smaller tools
    pass
```

## 🎓 What You Learned

✅ **@tool Decorator**: Simple way to create custom tools

✅ **Type Hints**: Required for LLM to understand parameters

✅ **Tool Descriptions**: Clear descriptions help LLM choose the right tool

✅ **Multiple Parameters**: Support various types (str, int, float, bool, list, dict)

✅ **Optional Parameters**: Use default values for optional params

✅ **Error Handling**: Return errors in result dict (don't raise exceptions)

✅ **Registration**: Use `register_tools()` to add custom tools to client

✅ **Best Practices**: Clear descriptions, type hints, error handling, structured returns

## 🚀 Next Steps

You've mastered custom tool creation! Now let's explore the powerful filesystem and code execution tools.

➡️ Continue to [06-filesystem-code-execution.ipynb](./06-filesystem-code-execution.ipynb) to learn how to:
- Execute Python code dynamically with `execute_python`
- Perform file operations with `filesystem_operation`
- Understand working directories and paths
- Combine computation and I/O operations
- Build a file-based counter application
- Handle security considerations for code execution